In [0]:
from pyspark.sql.functions import *

spark.sql("USE CATALOG real_time_projects")
spark.sql("USE SCHEMA realtime_log_analytics_alerting")

gold_df = (
    spark.readStream.table("enriched_logs")
    .withWatermark("timestamp", "10 minutes")
    .groupBy(
        window("timestamp", "5 minutes"),
        "service",
        "level",
        "tier"
    )
    .agg(
        count("*").alias("log_count"),
        avg("response_time_ms").alias("avg_response_time")
    )
)

gold_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/real_time_projects/realtime_log_analytics_alerting/project_volume/checkpoints/gold") \
    .outputMode("append") \
    .trigger(availableNow=True).table("gold_log_metrics")
